In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import re
from os import listdir
from os.path import isfile, join

In [ ]:
idx = pd.IndexSlice

<a id='opportunities_infos'></a>
# Opportunities infos by Created Date

In [ ]:
op_infos = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Opp_by_Created_Date.csv", encoding='latin-1', sep=";")

op_infos[["Created Date","Close Date", "Original Close Date", "Last Stage Change Date", "Last Modified Date"]] = op_infos.loc[:,["Created Date","Close Date", "Original Close Date", "Last Stage Change Date", "Last Modified Date"]].astype(str)
op_infos["Created Date"] = pd.to_datetime(op_infos["Created Date"], format="%d/%m/%Y")
op_infos["Close Date"] = pd.to_datetime(op_infos["Close Date"], format="%d/%m/%Y")
op_infos["Original Close Date"] = pd.to_datetime(op_infos["Original Close Date"], format="%d/%m/%Y")
op_infos["Last Stage Change Date"] = pd.to_datetime(op_infos["Last Stage Change Date"], format="%d/%m/%Y")
op_infos["Last Modified Date"] = pd.to_datetime(op_infos["Last Modified Date"], format="%d/%m/%Y")

columns = [c for c in op_infos.columns if not "Currency" in c]
op_infos = op_infos.loc[:,columns]

op_infos.loc[:, ["Stage Duration", "Age"]] = op_infos.loc[:, ["Stage Duration", "Age"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)
op_infos.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]] = op_infos.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)


In [ ]:
op_infos["Opp. Name freq"] = op_infos.groupby("Opportunity Name")["Opportunity Name"].transform("count")
op_infos["Opp. Incr. Amount"] = op_infos["Incremental Amount"] / op_infos["Opp. Name freq"]

In [ ]:
op_infos["Stage cat"] = pd.Categorical(op_infos["Stage"], ["Qualification","Needs Analysis","Solution Presentation / Demo", "Proposal", "Negotiation","Closed Won", "Closed Lost"])

In [ ]:
owner_role = ['IT North West Sales Team', 'IT North East Sales Team', 'IT Enterprise Specialists Sales Team', 'IT Central & South Sales Team', 'IT Large Account Sales Team']
filt = op_infos["Owner Role"].isin(owner_role)
op_infos_sintesi = op_infos.loc[filt,:].sort_values(['Stage cat']).groupby(["Stage cat", "Owner Role"]).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity Name":"count", "Incremental Amount":"sum", "Opp. Incr. Amount":"sum"}).rename(columns={"Opportunity Name":"N. Opportunità"}).unstack(level=1)
op_infos_sintesi.index.rename("Stage", inplace=True)
op_infos_sintesi_open = op_infos_sintesi.loc[idx["Closed Won":"Closed Lost"], idx[["Age", "N. Opportunità", "Incremental Amount", "Opp. Incr. Amount"],["IT North East Sales Team"]]]

op_infos_sintesi_open.style.format("{:,.0f}").background_gradient()

In [ ]:
op_infos_sintesi_closed = op_infos_sintesi.loc[idx["Closed Won":"Closed Lost"], idx[["Age", "N. Opportunità", "Incremental Amount"],["IT North East Sales Team"]]]
op_infos_sintesi_closed.style.format("{:,.0f}").background_gradient()

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\op_infos_sintesi2.xlsx", engine="openpyxl") as writer:
    op_infos_sintesi_closed.reset_index().to_excel(writer, sheet_name="Foglio1", index=False)
    op_infos_sintesi_open.reset_index().to_excel(writer, sheet_name="Foglio2", index=False)


<a id='win_rate'></a>
## Win rate

[Vai all'indice](#top)  

In [ ]:
op_infos["Month Op. Created"] = op_infos.groupby([op_infos["Created Date"].astype('datetime64[M]')])["Opportunity Name"].transform("count")

In [ ]:
op_infos_gr = op_infos.groupby([op_infos["Created Date"].astype('datetime64[M]'), op_infos["Close Date"].astype('datetime64[M]'), "Stage"]).agg({"Opportunity Name":"count", "Month Op. Created":"max", "Incremental Amount":"sum", "Stage Duration":"mean", "Age":"mean"})

op_infos_gr["% Stage Rate"] = (op_infos_gr["Opportunity Name"] / op_infos_gr["Month Op. Created"])

In [ ]:
'''op_infos_gr.index = op_infos_gr.index.set_levels([op_infos_gr.index.levels[0].astype('datetime64[M]'), op_infos_gr.index.levels[1].astype('datetime64[M]'), op_infos_gr.index.levels[2]])'''

In [ ]:
op_infos_gr_exp = op_infos_gr.copy()
op_infos_gr_exp.index = op_infos_gr_exp.index.set_levels([op_infos_gr_exp.index.levels[0].strftime('%B-%Y'), op_infos_gr_exp.index.levels[1].strftime('%B-%Y'), op_infos_gr_exp.index.levels[2]])
op_infos_gr_exp.loc[idx[:,:,["Closed Lost", "Closed Won"]], idx["Opportunity Name","Incremental Amount", "% Stage Rate", "Age"]].rename(columns={'Opportunity Name':'Opportunities'}).style.format({"Incremental Amount":"{:,.0f}", "% Stage Rate": "{:.2%}", "Age":"{:.0f}"}).background_gradient(subset=["% Stage Rate"])

#.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Win_rate.xlsx")

***

In [ ]:
op_infos_gr_exp.reset_index().to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\IT_Opp_infos.xlsx", index=False)

***

<a id='win_rate'></a>
### Insert a total row on groupby

In [ ]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
               'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
               'C' : np.random.randn(8),
               'D' : np.random.randn(8)})

df.B = pd.Categorical(df.B, 
                      categories=np.append(df.B.unique(), 'Total'))

v = df.groupby(by=['A', 'B']).sum()

v.loc(axis=0)[pd.IndexSlice[:,'Total']] = v.groupby(level=0).sum().values

In [ ]:
op_infos["Stage"] = pd.Categorical(op_infos["Stage"], 
                      categories=np.append(op_infos["Stage"].unique(), 'Total'))

prova_gr = op_infos.groupby([op_infos["Created Date"].astype('datetime64[M]'), op_infos["Close Date"].astype('datetime64[M]'), "Stage"]).agg({"Opportunity Name":"count", "Month Op. Created":"max", "Incremental Amount":"sum"})

prova_gr["% Stage Rate"] = (prova_gr["Opportunity Name"] / prova_gr["Month Op. Created"])

prova_gr.loc(axis=0)[idx[:,:,"Total"]] = prova_gr.groupby(["Created Date", "Close Date"]).sum().values


prova_gr.index = prova_gr.index.set_levels([prova_gr.index.levels[0].strftime('%B-%Y'), prova_gr.index.levels[1].strftime('%B-%Y'), prova_gr.index.levels[2]])

def df_style(val):
    return "font-weight: bold"

prova_gr.loc[idx[:,:,["Closed Lost", "Closed Won", "Total"]], idx["Opportunity Name","Incremental Amount", "% Stage Rate"]].rename(columns={'Opportunity Name':'Opportunities'}).style.format({"Opportunities":"{:,.0f}","Incremental Amount":"{:,.0f}", "% Stage Rate": "{:.2%}"}).background_gradient(subset=["% Stage Rate"]).applymap(df_style, subset=idx[:,:,"Total"])

<a id='leads_and_opportunities'></a>
# Leads and opportunities

In [ ]:
lead_op = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Leads_w_conversion.csv", encoding='latin-1', sep=";")

In [ ]:
date_columns = [c for c in lead_op.columns if " Date" in c]

for col in date_columns:
    lead_op[col] = lead_op[col].apply(lambda x: pd.to_datetime(str(x), format="%d/%m/%Y"))
    
    
columns = [c for c in lead_op.columns if not "Currency" in c]
lead_op = lead_op.loc[:,columns]

lead_op.dropna(axis=1, how="all", inplace=True)

In [ ]:
lead_op["MQL Duration"] = lead_op["MQL Duration"].str.split(" ", expand=True).get(0).fillna(0).astype(np.int64)
lead_op["SAL Duration"] = lead_op["SAL Duration"].str.split(" ", expand=True).get(0).fillna(0).astype(np.int64)

In [ ]:
lead_op.info()

In [ ]:
pd.set_option('display.max_rows', 30)
lead_op.loc[(lead_op["Lead Status"].isin(["SQL"])) & (lead_op["Created Date"].dt.year ==2022) ,["Owner Role Name","Lead Owner","Lead Source","Email","Company / Account","Lead Status", "Created Date", "Lead Age", "MQL Duration", "SAL Duration", "Converted Date", "Opportunity: Created Date","Opportunity Name"]]

In [ ]:
lead_op['Lead Status cat'] = pd.Categorical(lead_op['Lead Status'], ["New", "MQL", "SAL", "SQL", "Rejected", "Qualified", "Unqualified", "Working"])

In [ ]:
g_status = lead_op.sort_values(['Lead Status cat']).groupby(["Lead Status cat"])

In [ ]:
lead_sintesi = g_status[["Lead ID", "Opportunity ID", "Lead Age","MQL Duration","SAL Duration"]].agg({"Lead ID":"nunique", "Opportunity ID":"nunique", "Lead Age":"mean","MQL Duration":"mean","SAL Duration":"mean"})\
.reset_index().rename(columns={"Lead Status cat":"Lead Status", "Lead ID":"N.Lead", "Opportunity ID":"N.Opportunità"})\
.style.format({"N.Lead":"{:,.0f}", "N.Opportunità": "{:,.0f}", "CR":"{:.2%}", "Lead Age":"{:.0f} giorni","MQL Duration":"{:.0f} giorni","SAL Duration":"{:.0f} giorni"}).background_gradient()

lead_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\Lead_status_sintes.xlsx", index=False)

#.merge(g_status[["Lead ID", "Opportunity ID"]].apply(lambda x: pd.Series({'CR': x['Opportunity ID'].nunique() / x['Lead ID'].nunique()})), left_index=True, right_index=True)

In [ ]:
g_source = lead_op.groupby(["Lead Source"], dropna=False)

In [ ]:
g_source["Lead ID"].nunique().to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\source.xlsx")

In [ ]:
g_busneed = lead_op.groupby(["Business need"], dropna=False)

In [ ]:
lead_op.groupby(["Business need"])["Lead ID"].nunique().to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\buz_needs.xlsx")

In [ ]:
g_age = lead_op.groupby(["Lead Age"])

In [ ]:
tm = ["IT"]
#tm =["IT Central & South Sales Team","IT North East Sales Team","IT North West Sales Team","IT Enterprise Specialists Sales Team"]
status = ["MQL","SAL","SQL","Rejected"]
filt = (lead_op["Owner Role Name"].isin(tm) & lead_op["Lead Status"].isin(status) )

lead_op.loc[filt, ["Lead Status", "Owner Role Name", "Lead Owner", "Lead ID"]].groupby(["Lead Status", "Owner Role Name", "Lead Owner"]).count().unstack(level=(1,2)).head(6)
#.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\Lead_age_per_owner.xlsx")

In [ ]:
lead_op.info()

***

# IT_Op_w_Products

In [3]:
op_w_prod = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Op_w_Products.csv", encoding='latin-1', sep=";")

op_w_prod["Close Date"] = op_w_prod["Close Date"].astype(str)
op_w_prod["Close Date"] = pd.to_datetime(op_w_prod["Close Date"], format="%d/%m/%Y")
op_w_prod["Created Date"] = op_w_prod["Created Date"].astype(str)
op_w_prod["Created Date"] = pd.to_datetime(op_w_prod["Created Date"], format="%d/%m/%Y")

columns = [c for c in op_w_prod.columns if not "Currency" in c]
op_w_prod = op_w_prod[columns]

op_w_prod.loc[:, ["Stage Duration", "Age"]] = op_w_prod.loc[:, ["Stage Duration", "Age"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)
op_w_prod.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount","Total Value","Total Price"]] = op_w_prod.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount","Total Value","Total Price"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

op_w_prod["Product Name"].fillna("-", inplace=True)
op_w_prod["Products"] = op_w_prod.groupby("Opportunity ID")["Product Name"].transform(lambda x: " | ".join(x))

In [4]:
op_w_prod["Opp. ID freq"] = op_w_prod.groupby("Opportunity ID")["Opportunity ID"].transform("count")
op_w_prod["Opp. Incr. Amount"] = op_w_prod["Incremental Amount"] / op_w_prod["Opp. ID freq"]

In [5]:
op_w_prod["Stage cat"] = pd.Categorical(op_w_prod["Stage"], ["Qualification","Needs Analysis","Solution Presentation / Demo", "Proposal", "Negotiation","Closed Won", "Closed Lost"])

## Opportunity by stage

### Opportunity by stage by owner role

In [9]:
owner_role = ['IT North West Sales Team', 'IT North East Sales Team', 'IT Enterprise Specialists Sales Team', 'IT Central & South Sales Team', 'IT Large Account Sales Team']
filt = op_w_prod["Owner Role"].isin(owner_role)

op_by_stage_owner_role = op_w_prod.loc[filt,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat"]).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity ID":"nunique", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity ID":"N. Opportunità"})

op_by_stage_owner_role.index.rename(("Owner Role","Stage"), inplace=True)

### Opportunity by stage

In [10]:
op_by_stage = op_w_prod.sort_values(['Stage cat']).groupby(["Stage cat"]).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity ID":"nunique", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity ID":"N. Opportunità"})

op_by_stage.index.rename(("Stage"), inplace=True)

## Velocity

#### Velocity by stage by owner role - Opportunità aperte

In [11]:
velocity_stage_owner_role_open = op_by_stage_owner_role.loc[idx[:, ["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["Stage Duration", "N. Opportunità"]]

velocity_stage_owner_role_open.style.format("{:,.0f}").background_gradient()


#### Velocity by stage by owner role - Opportunità chiuse

In [15]:
velocity_stage_owner_role_closed = op_by_stage_owner_role.loc[idx[:, ["Closed Won", "Closed Lost"]], idx["Age", "N. Opportunità"]]

velocity_stage_owner_role_closed.style.format("{:,.0f}").background_gradient()


#### Velocity by stage - Opportunità aperte

In [13]:
velocity_stage_open = op_by_stage.loc[idx[["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["Stage Duration", "N. Opportunità"]]

velocity_stage_open.style.format("{:,.0f}").background_gradient()


,Stage Duration,N. Opportunità
Stage,,
Qualification,134,556
Needs Analysis,80,14
Solution Presentation / Demo,139,555
Proposal,166,902
Negotiation,193,81


#### Velocity by stage - Opportunità chiuse

In [14]:
velocity_stage_closed = op_by_stage.loc[idx[ ["Closed Won", "Closed Lost"]], idx["Age", "N. Opportunità"]]

velocity_stage_closed.style.format("{:,.0f}").background_gradient()


,Age,N. Opportunità
Stage,,
Closed Won,18,"11,245"
Closed Lost,110,"3,724"


## Volume

#### Volume by stage by owner role

In [18]:
volume_stage_owner_role = op_by_stage_owner_role.loc[idx[:, ["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["N. Opportunità", "Opp. Incr. Amount"]]

volume_stage_owner_role.style.format("{:,.0f}").background_gradient()


#### Volume by stage

In [22]:
volume_stage = op_by_stage.loc[idx[["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["N. Opportunità", "Opp. Incr. Amount"]]

volume_stage.style.format("{:,.0f}").background_gradient()


,N. Opportunità,Opp. Incr. Amount
Stage,,
Qualification,556,"1,205,545"
Needs Analysis,14,"238,090"
Solution Presentation / Demo,555,"1,090,039"
Proposal,902,"1,132,285"
Negotiation,81,"133,389"


In [36]:
volume_stage_by_date = op_w_prod.sort_values(['Stage cat']).groupby(["Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"}).unstack(level=1)    

volume_stage_by_date.style.format("{:,.0f}").background_gradient()


## Export to excel

In [33]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage.xlsx", engine="openpyxl") as writer:
    velocity_stage_open.to_excel(writer, sheet_name="vel_open")
    velocity_stage_owner_role_open.to_excel(writer, sheet_name="vel_open_dett")
    velocity_stage_closed.to_excel(writer, sheet_name="vel_closed")
    velocity_stage_owner_role_closed.to_excel(writer, sheet_name="vel_closed_dett")
    volume_stage.to_excel(writer, sheet_name="vol_open")
    volume_stage_owner_role.to_excel(writer, sheet_name="vol_open_dett")
    volume_stage_by_date.to_excel(writer, sheet_name="vol_by_date")

---

### N. of opportunity by stage and created date

In [29]:
qualification = op_w_prod.loc[(op_w_prod["Stage cat"] == "Qualification") & (op_w_prod["Stage Duration"] > 7),:]
analysis = op_w_prod.loc[(op_w_prod["Stage cat"] == "Needs Analysis") & (op_w_prod["Stage Duration"] > 7),:]
demo = op_w_prod.loc[(op_w_prod["Stage cat"] == "Solution Presentation / Demo") & (op_w_prod["Stage Duration"] > 30) & (op_w_prod["Close Date"].dt.month == dt.datetime.now().month),:]
proposal = op_w_prod.loc[(op_w_prod["Stage cat"] == "Proposal") & (op_w_prod["Stage Duration"] > 15),:]
negotiation = op_w_prod.loc[(op_w_prod["Stage cat"] == "Negotiation") & (op_w_prod["Stage Duration"] > 35),:]

op_w_prod_filt = pd.concat([qualification,analysis,demo,proposal,negotiation], axis=0)

op_w_prod_filt_gr = op_w_prod_filt.sort_values(['Stage cat']).groupby(["Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"}).unstack(level=1)    

op_w_prod_filt_gr.loc[idx[["Qualification", "Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], :]

Opportunity ID                                   \
Created Date                     2020-10-01 2020-11-01 2020-12-01 2021-01-01   
Stage cat                                                                      
Qualification                             5         23          2          3   
Needs Analysis                            0          1          0          0   
Solution Presentation / Demo              2         10          3          3   
Proposal                                 24         19         11         10   
Negotiation                               1          6          0          1   

                                                                          \
Created Date                 2021-02-01 2021-03-01 2021-04-01 2021-05-01   
Stage cat                                                                  
Qualification                         7         19         19         17   
Needs Analysis                        0          0          0          0   
Solution Presentation / Demo          7         15         13          9   
Proposal                             10         40         52         37   
Negotiation                           0          3          3          1   

                                                    ... Opp. Incr. Amount  \
Created Date                 2021-06-01 2021-07-01  ...        2021-07-01   
Stage cat                                           ...                     
Qualification                         6          8  ...          81500.00   
Needs Analysis                        0          0  ...              0.00   
Solution Presentation / Demo         14          7  ...          30600.00   
Proposal                             26         10  ...          20517.27   
Negotiation                           1          2  ...           4125.00   

                                                                          \
Created Date                 2021-08-01 2021-09-01 2021-10-01 2021-11-01   
Stage cat                                                                  
Qualification                       0.0    5125.00    4590.00     9620.0   
Needs Analysis                      0.0       0.00  100000.00        0.0   
Solution Presentation / Demo     1000.0   34380.00   53490.00    60285.0   
Proposal                            0.0  114817.42   51711.11    60107.0   
Negotiation                         0.0    5190.00    3680.00      550.0   

                                                                          \
Created Date                 2021-12-01 2022-01-01 2022-02-01 2022-03-01   
Stage cat                                                                  
Qualification                   54150.0     9889.0    15074.0   71852.33   
Needs Analysis                  50350.0        0.0        0.0       0.00   
Solution Presentation / Demo    63515.0    96955.0    61890.0   88900.00   
Proposal                        40862.0    53613.0    88348.0  162694.00   
Negotiation                       450.0     5715.0    14478.0   19074.00   

                                         
Created Date                 2022-04-01  
Stage cat                                
Qualification                    5555.0  
Needs Analysis                      0.0  
Solution Presentation / Demo        0.0  
Proposal                        67377.0  
Negotiation                         0.0  

[5 rows x 38 columns]

In [30]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_date_and_stage_duration.xlsx", engine="openpyxl") as writer:
    qualification.to_excel(writer, sheet_name="qualification")
    analysis.to_excel(writer, sheet_name="analysis")
    demo.to_excel(writer, sheet_name="demo")
    proposal.to_excel(writer, sheet_name="proposal")
    negotiation.to_excel(writer, sheet_name="negotiation")
    op_w_prod_filt_gr.to_excel(writer, sheet_name="op_w_prod_filt_gr")

### Opportunity by stage and created date

In [ ]:
op_w_prod_sintesi = op_w_prod.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opp. Incr. Amount":"sum"}).unstack(level=2)

op_w_prod_sintesi.index.rename(("Owner Role","Stage"), inplace=True)

op_w_prod_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_date.xlsx")
op_w_prod_sintesi

### Opportunity by stage and close date

In [ ]:
op_w_prod_sintesi = op_w_prod.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat",op_w_prod["Close Date"].astype("datetime64[M]")]).agg({"Opp. Incr. Amount":"sum"}).unstack(level=2)

op_w_prod_sintesi.index.rename(("Owner Role","Stage"), inplace=True)

op_w_prod_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_close_date.xlsx")
op_w_prod_sintesi

### Opportunity by stage and product

In [ ]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [ ]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod.loc[:, "Codice PRODOTTO"].apply(lambda x: "IT-" + x.zfill(8))

In [ ]:
op_w_prod_merged = op_w_prod.merge(cod_prod, how="left", left_on="Product Code", right_on="Codice PRODOTTO")

In [ ]:
op_w_prod_merged_sintesi = op_w_prod_merged.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","MDM","Stage cat"], dropna=False).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity Name":"count", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity Name":"N. Opportunità"})

op_w_prod_merged_sintesi.index.rename(("Owner Role","Product","Stage"), inplace=True)

op_w_prod_merged_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_prod.xlsx")

op_w_prod_merged_sintesi

In [ ]:
op_w_prod_merged_sintesi = op_w_prod_merged.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","MDM","Stage cat"], dropna=False).agg({"Stage Duration":"mean"})
op_w_prod_merged_sintesi.index.rename(("Owner Role","Product","Stage"), inplace=True)
op_w_prod_merged_sintesi.unstack(level=1).to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_prod.xlsx", )


In [ ]:
op_w_prod_merged_sintesi.unstack(level=1)